In [1]:
using EnvelopeApproximation
using EnvelopeApproximation.BubbleBasics
using EnvelopeApproximation.BubblesEvolution
using StableRNGs
using Base.Iterators
using Measurements
using JLD2
using Plots
using EnvelopeApproximation.Visualization
import EnvelopeApproximation.BubblesEvolution.sample
ENV["JULIA_DEBUG"] = EnvelopeApproximation

EnvelopeApproximation

In [2]:
β = 1.
Δt = β / 100
eg = ExponentialGrowth(β, Δt, Γ_0 = 1.)
bs = BallSpace(1., Point3(0., 0., 0.))

BallSpace(1.0, Point3([0.0, 0.0, 0.0]))

In [3]:
N = 100
η = 0.8
rng = StableRNG(1)
function ts(state, space, _):: Bool
    ps = sample(rng, N, space)
    cbs = current_bubbles(state)
    length(cbs) == 0. && return false
    inside = sum([p ∈ cbs for p in ps])
    return inside / N ≥ η 
end

ts (generic function with 1 method)

# Dependence on $\Delta t$

In [4]:
Δts = 10 .^ (range(-3., 0., 20))
egs = ExponentialGrowth.(β, Δts, Γ_0 = 1.) 
rngs = StableRNG.(1:30);

In [5]:
_evolve(eg, rng) = evolve(eg, bs, termination_strategy=ts, rng=rng)
_evolve(t:: Tuple) = _evolve(t...)

_evolve (generic function with 2 methods)

In [6]:
evolves = _evolve.(product(egs, rngs))

┌ Info: Initiating PT of BubblesSnapShot(@NamedTuple{time::Float64, site::Point3}[], 0.0, EnvelopeApproximation.BubblesEvolution.speed_of_light_profile)
└ @ EnvelopeApproximation.BubblesEvolution /home/ben/.julia/dev/EnvelopeApproximation/src/BubblesEvolution.jl:116
┌ Debug: A total of 0 nucleations was sampled in accordance with the expected mean of 0.0010005001667083846
└ @ EnvelopeApproximation.BubblesEvolution /home/ben/.julia/dev/EnvelopeApproximation/src/BubblesEvolution.jl:94


MethodError: MethodError: no method matching +(::Point3, ::Vector{StaticArraysCore.SVector{3, Float64}})
The function `+` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:596
  +(!Matched::ChainRulesCore.NotImplemented, ::Any)
   @ ChainRulesCore ~/.julia/packages/ChainRulesCore/6Pucz/src/tangent_arithmetic.jl:24
  +(::Any, !Matched::ChainRulesCore.ZeroTangent)
   @ ChainRulesCore ~/.julia/packages/ChainRulesCore/6Pucz/src/tangent_arithmetic.jl:100
  ...


## Mean number of nucleations

In [7]:
mean(x) = sum(x) / length(x)
std(x) = sqrt(sum(x .^ 2) .- mean(x) ^ 2) / (sqrt(length(x) * (length(x) - 1)))
average(x) = mean(x) ± std(x) 
mean_nucleations(x) = average((z -> length(z.nucleations)).(x))

mean_nucleations (generic function with 1 method)

In [8]:
nucs = mean_nucleations.(eachrow(evolves))

UndefVarError: UndefVarError: `evolves` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [9]:
custom_xticks = 10. .^(-3, -2, -1, 0)
custom_xticks_strs = ["$f" for f in custom_xticks]
scatter(Δts, nucs, xscale=:log10, xticks=(custom_xticks, custom_xticks_strs), xlabel="Δt [β]", ylabel="N")

UndefVarError: UndefVarError: `scatter` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing MakieCore in the current active module Main
Hint: a global variable of this name may be made accessible by importing Makie in the current active module Main
Hint: a global variable of this name may be made accessible by importing GLMakie in the current active module Main

## Mean PT time

In [10]:
pt_time(x) = average((z -> z.t).(x))

pt_time (generic function with 1 method)

In [11]:
pt_times = pt_time.(eachrow(evolves))

UndefVarError: UndefVarError: `evolves` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [12]:
scatter(Δts, pt_times, xscale=:log10, xticks=(custom_xticks, custom_xticks_strs), xlabel="Δt [β]", ylabel="T [β]")

UndefVarError: UndefVarError: `scatter` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing MakieCore in the current active module Main
Hint: a global variable of this name may be made accessible by importing Makie in the current active module Main
Hint: a global variable of this name may be made accessible by importing GLMakie in the current active module Main

# Study dependence on Space Size

In [13]:
radii = range(1., 10., 10)

1.0:1.0:10.0

In [14]:
max_R = radii[end]

10.0

In [15]:
function _evolve(radius:: Float64, rng)
    eg = ExponentialGrowth(β, Δt, Γ_0 = (4π/3) * (radius / max_R) ^3)
    bs = BallSpace(radius, Point3(0., 0., 0.))
    return evolve(eg, bs, termination_strategy=ts, rng=rng)
end

_evolve(t:: Tuple) = _evolve(t...)

_evolve (generic function with 3 methods)

In [16]:
evolves = _evolve.(product(radii, rngs))

UndefVarError: UndefVarError: `product` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing IterTools in the current active module Main

In [17]:
using EnvelopeApproximation.BubblesIntegration

UndefVarError: UndefVarError: `BubblesIntegration` not defined in `EnvelopeApproximation`
Suggestion: check for spelling errors or missing imports.

In [18]:
viz(evolves[10, 1])

UndefVarError: UndefVarError: `viz` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing Meshes in the current active module Main
Hint: a global variable of this name may be made accessible by importing MeshesMakieExt in the current active module Main

In [19]:
total_vol = (4π / 3) * radii[10] ^ 3

4188.790204786391

In [20]:
vol = volume_integral(x -> 1., current_bubbles(evolves[10, 7]), 1. / (3 * 10), (2π / 10.), (2. / 10))

UndefVarError: UndefVarError: `volume_integral` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [21]:
vol / total_vol

UndefVarError: UndefVarError: `vol` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Nucleations num

In [22]:
nucs = mean_nucleations.(eachrow(evolves))

UndefVarError: UndefVarError: `evolves` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [23]:
nucs

UndefVarError: UndefVarError: `nucs` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [24]:
scatter(radii, nucs, xlabel="r [β]", ylabel="N")

UndefVarError: UndefVarError: `scatter` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing MakieCore in the current active module Main
Hint: a global variable of this name may be made accessible by importing Makie in the current active module Main
Hint: a global variable of this name may be made accessible by importing GLMakie in the current active module Main

## Mean PT time

In [25]:
pt_time(x) = average((z -> z.t).(x))

pt_time (generic function with 1 method)

In [26]:
pt_times = pt_time.(eachrow(evolves))

UndefVarError: UndefVarError: `evolves` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [27]:
scatter(radii, pt_times)

UndefVarError: UndefVarError: `scatter` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing MakieCore in the current active module Main
Hint: a global variable of this name may be made accessible by importing Makie in the current active module Main
Hint: a global variable of this name may be made accessible by importing GLMakie in the current active module Main

# Saving the Data

In [28]:
jldsave("evolution_ensemble.jld2"; space_size=evolves)

UndefVarError: UndefVarError: `evolves` not defined in `Main`
Suggestion: check for spelling errors or missing imports.